## Credit Risk Modeling

* Analyze & Breakdown german credit Portfolio to learn about possible trends
 * POC Credit Scoring model 
   * **Project Goal:** The credit risk application scorecard would assist in evaluating probability of customer defaults. This end-to-end advanced credit risk analytics would focuses on:
     * Meeting business goals 
     * Optimization of the overall customer acquisition funnel
     * Build a holistic customer risk profile
  ##### PART 1 Data Preparation and Exporatory Data Analysis

  * Data Audit: Verifying data quality
  * Distribution of categories for each variable
  * Trend of variable vs default 


    
 ##### PART 2 Feature Engineering
  * Cross variables
  * Ratios

  ##### PART 3 Characteristics Analysis
  * Handle missing values
  * Characteristics Analysis Report
    * Fine Classing 
    * Coarse Classing 
    * Information value 
    * WoE transformations 
    * Univariate Analysis/ Feature Selection
  
 ##### PART 4 Scorecard Development
  * Modelling Phase
   * Variable Selection
  * Model Correlation Analysis
  * Model evaluation: 
     * Hold Out Sample Validation


In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt


#import utility file
%run utils.ipynb

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Util Libraries successfully imported


In [2]:
#read data
data =pd.read_csv('GermanCredit.csv')
data.head()

,Unnamed: 0,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,residence_history,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender
0,0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,none,6 years,real estate,67,none,own,2,0,1,2.349340e+09,yes,skilled employee,male
1,1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,none,5 months,real estate,22,none,own,1,1,1,NaN,yes,skilled employee,female
2,2,NaN,12,critical,education,2096,24.0,5 years,2,single,none,4 years,real estate,49,none,own,1,0,2,NaN,yes,unskilled resident,male
3,3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,guarantor,13 years,building society savings,45,none,for free,1,0,2,NaN,yes,skilled employee,male
4,4,-23.0,24,delayed,car (new),4870,43.0,3 years,3,single,none,13 years,unknown/none,53,none,for free,2,1,2,NaN,yes,skilled employee,male


### Data Audit

The purpose of data auditing is to assess the quality, accuracy, completeness, and reliability of data 
The quality of the available data is an important element in the development of an effective model. The first step is therefore to perform a qualitative inspection using frequency tables.

In [272]:

audit(data)

,Variable,No of Unique,No of Missing,% Missing,No of Zeros,% Zeros,No of Rows
0,Unnamed: 0,1000,0,0.000,1,0.001,1000
1,checking_balance,263,394,0.394,0,0.000,1000
2,months_loan_duration,33,0,0.000,0,0.000,1000
3,credit_history,5,0,0.000,0,0.000,1000
4,purpose,10,0,0.000,0,0.000,1000
5,amount,921,0,0.000,0,0.000,1000
6,savings_balance,302,183,0.183,8,0.008,1000
7,employment_length,31,62,0.062,0,0.000,1000
8,installment_rate,4,0,0.000,0,0.000,1000
9,personal_status,3,310,0.310,0,0.000,1000


### INITIAL DATA EXPLORATION

In [273]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1000 non-null   int64  
 1   checking_balance      606 non-null    float64
 2   months_loan_duration  1000 non-null   int64  
 3   credit_history        1000 non-null   object 
 4   purpose               1000 non-null   object 
 5   amount                1000 non-null   int64  
 6   savings_balance       817 non-null    float64
 7   employment_length     938 non-null    object 
 8   installment_rate      1000 non-null   int64  
 9   personal_status       690 non-null    object 
 10  other_debtors         1000 non-null   object 
 11  residence_history     870 non-null    object 
 12  property              1000 non-null   object 
 13  age                   1000 non-null   int64  
 14  installment_plan      1000 non-null   object 
 15  housing               

In [276]:
# Distribution of Purpose Attributes by application volume
data['purpose'].value_counts()
# Output=False


radio/tv               280
car (new)              234
furniture              181
car (used)             103
business                97
education               50
repairs                 22
domestic appliances     12
others                  12
retraining               9
Name: purpose, dtype: int64

In [3]:
#group purpose.
# Try to reduce the categories in a variable as much as possible. categories like with less than 5% proportion of total can be grouped together
data['purpose_grped'] = np.where(data['purpose'].isin(['domestic appliances','radio/tv' ]), 'domestic appliance', 
                        np.where(data['purpose'].isin(['car (new)', 'car (used)']), 'Car',
                        np.where(data['purpose'].isin(['others','retraining', 'repairs' ]), 'Others',    data['purpose'])))
data['purpose_grped'].value_counts()

Car                   337
domestic appliance    292
furniture             181
business               97
education              50
Others                 43
Name: purpose_grped, dtype: int64

In [4]:
# Distribution of personal_status Attributes by application volume
data['personal_status'].value_counts()
# Output=False

single      548
married      92
divorced     50
Name: personal_status, dtype: int64

In [5]:
# Distribution of other_debtors Attributes by application volume
data['other_debtors'].value_counts()
# Output=False

none            907
guarantor        52
co-applicant     41
Name: other_debtors, dtype: int64

In [6]:
# Distribution of residence_history Attributes by application volume
data['residence_history'].value_counts()

Output=False

In [9]:
# Distribution of property Attributes by application volume
data['property'].value_counts()
# Output=False

other                       332
real estate                 282
building society savings    232
unknown/none                154
Name: property, dtype: int64

In [10]:
# Distribution of residence_history Attributes by application volume
data['installment_plan'].value_counts()
# Output=False

none      814
bank      139
stores     47
Name: installment_plan, dtype: int64

In [11]:
# Distribution of housing Attributes by application volume
data['housing'].value_counts()
# Output=False

own         713
rent        179
for free    108
Name: housing, dtype: int64

In [12]:
# Distribution of dependents Attributes by application volume
data['dependents'].value_counts()
# Output=False

1    845
2    155
Name: dependents, dtype: int64

In [13]:
# Distribution of gender Attributes by application volume
data['gender'].value_counts()
# Output=False

male      690
female    310
Name: gender, dtype: int64

In [14]:
# Distribution of gender Attributes by application volume
data['job'].value_counts()
# Output=False

skilled employee           630
unskilled resident         200
mangement self-employed    148
unemployed non-resident     22
Name: job, dtype: int64

### ANALYSIS

check the default distribution across input variables

#### Credit history

In [15]:

ES1 = pd.crosstab(index=data['credit_history'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
credit_history,,,
fully repaid,15,25,62.50
fully repaid this bank,21,28,57.14
repaid,361,169,31.89
delayed,60,28,31.82
critical,243,50,17.06


#### Job

In [16]:

ES1 = pd.crosstab(index=data['job'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
job,,,
mangement self-employed,97,51,34.46
unemployed non-resident,15,7,31.82
skilled employee,444,186,29.52
unskilled resident,144,56,28.00


#### Purpose

In [17]:

ES1 = pd.crosstab(index=data['purpose'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
purpose,,,
education,28,22,44.00
others,7,5,41.67
car (new),145,89,38.03
repairs,14,8,36.36
business,63,34,35.05
domestic appliances,8,4,33.33
furniture,123,58,32.04
radio/tv,218,62,22.14
car (used),86,17,16.50


#### Personal status

In [18]:

ES1 = pd.crosstab(index=data['personal_status'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
personal_status,,,
divorced,30,20,40.00
married,67,25,27.17
single,402,146,26.64


#### Property

In [19]:

ES1 = pd.crosstab(index=data['property'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
property,,,
unknown/none,87,67,43.51
other,230,102,30.72
building society savings,161,71,30.60
real estate,222,60,21.28


#### Installment plan

In [20]:

ES1 = pd.crosstab(index=data['installment_plan'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
installment_plan,,,
bank,82,57,41.01
stores,28,19,40.43
none,590,224,27.52


#### Housing

In [21]:

ES1 = pd.crosstab(index=data['housing'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
housing,,,
for free,64,44,40.74
rent,109,70,39.11
own,527,186,26.09


#### Dependent

In [22]:

ES1 = pd.crosstab(index=data['dependents'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
dependents,,,
1,591,254,30.06
2,109,46,29.68


#### Foreign worker

In [23]:

ES1 = pd.crosstab(index=data['foreign_worker'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
foreign_worker,,,
yes,667,296,30.74
no,33,4,10.81


#### Job

In [24]:

ES1 = pd.crosstab(index=data['job'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
job,,,
mangement self-employed,97,51,34.46
unemployed non-resident,15,7,31.82
skilled employee,444,186,29.52
unskilled resident,144,56,28.00


#### Gender

In [25]:

ES1 = pd.crosstab(index=data['gender'],
            columns=data['default'])
ES1['Bad Rate(%)']= ES1[1]/ES1.sum(axis=1) *100
ES1.sort_values('Bad Rate(%)', ascending=False).round(2)

default,0,1,Bad Rate(%)
gender,,,
female,201,109,35.16
male,499,191,27.68


##### Numerical data analysis

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  1000 non-null   int64  
 1   checking_balance            606 non-null    float64
 2   months_loan_duration        1000 non-null   int64  
 3   credit_history              1000 non-null   object 
 4   purpose                     1000 non-null   object 
 5   amount                      1000 non-null   int64  
 6   savings_balance             817 non-null    float64
 7   employment_length           938 non-null    object 
 8   installment_rate            1000 non-null   int64  
 9   personal_status             690 non-null    object 
 10  other_debtors               1000 non-null   object 
 11  residence_history           870 non-null    object 
 12  property                    1000 non-null   object 
 13  age                         1000 n